# Web3 Scripts for Django
This notebook is used as my development environment for the scripts I intend to integrate with the LUCE Frontend. I prototype the functions here and then migrate them into script files to be used in conjunction with Django.


## First iteration:

### Prepare Connection to Blockchain


In [1]:
from web3 import Web3

In [2]:
!which python

/home/vagrant/miniconda/envs/luce_vm/bin/python


In [3]:
# Use Ganache as web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

### Create Ethereum Account

**Interactive**

In [28]:
# Create a new account
eth_account = w3.eth.account.create()

In [29]:
# Display address
eth_address = eth_account.address
eth_account.address

'0x71170a6d4400ee314AA88565a6751621497f72d3'

In [30]:
# Show private key
eth_pkey = eth_account.privateKey
eth_account.privateKey

HexBytes('0x0d5d7661123753be017e1341164742472596104c130b33313c14a2f7eba802b8')

In [106]:
# Show balance
w3.eth.getBalance(eth_address)

0

**Function**  
```
Inputs: -  
Output: account object (public key & private key)
```

In [69]:
# Create class to store address and private key
class EthAccount():
    address = None
    pkey = None

In [63]:
def create_wallet():
    eth_account = EthAccount()
    eth_account_raw = w3.eth.account.create()
    eth_account.address = eth_account_raw.address
    eth_account.pkey = eth_account_raw.privateKey
    return (eth_account)

In [66]:
eth_account = create_wallet()

### Fund Account

In [67]:
# Extract default accounts created by ganache
accounts = w3.eth.accounts

In [68]:
accounts

['0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA',
 '0xbf7ca7e607516Cd0F1b94a6C77F45fbFD936949C',
 '0x43e196C418b4b7Ebf71ba534042cC8907bd39dc9',
 '0x2aD1e0fAC9d6D523Fb1f21aE1Bc6acA09dD60c7F',
 '0x3c93550555cDD5218bB2923008b3Fb4ea6Ee4618',
 '0xf12C911960968c187090b52998a3597b35BC2619',
 '0x172bE1ee3f659A83a9C307fafafA51c9F91954Ae',
 '0x515f90440fA63377d7451E58B3be334F9393Ad74',
 '0x27F82fD9E21a3758737Cf3BCCA9652064186107C',
 '0x6eEA265cd899aE17Cf588744E4C7Fe31e3258E85']

**Store credentials of first Ganache account to use as faucet**

In [70]:
# Instantiate faucet object
faucet = EthAccount()

In [71]:
# Wallet address
faucet.address       = "0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA"

# Private key (from Ganache interface)
faucet.pkey   = "0x4a2cb86c7d3663abebf7ab86a6ddc3900aee399750f35e65a44ecf843ec39116"

**Send Function**

In [77]:
# Define a function to send ether
import time
def send_ether(amount_in_ether, recipient_address, sender_address = faucet.address, sender_pkey=faucet.pkey):
    amount_in_wei = w3.toWei(amount_in_ether,'ether');

    # How many transactions have been made by wallet?
    # This is required and prevents double-spending.
    # Different from nonce in block mining.
    nonce = w3.eth.getTransactionCount(sender_address)
    
    # Specify transcation dictionary
    txn_dict = {
            'to': recipient_address,
            'value': amount_in_wei,
            'gas': 2000000,
            'gasPrice': w3.toWei('40', 'gwei'),
            'nonce': nonce,
            'chainId': 3
    }
    
    # Sign transaction
    signed_txn = w3.eth.account.signTransaction(txn_dict, sender_pkey)

    # Send transaction & store transaction hash
    txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

    # Check if transaction was added to blockchain
    # time.sleep(0.5)
    txn_receipt = w3.eth.getTransactionReceipt(txn_hash)
    return txn_hash

In [78]:
# Set recipient
recipient = eth_account.address

In [79]:
# Send ether and store transaction hash
txn_hash = send_ether(1.5,recipient)

In [83]:
# Show balance
w3.eth.getBalance(eth_account.address)

1500000000000000000

## Deploy Contract

### Compile contract from file

In [86]:
# Import libraries
#import json
#import web3

#from web3 import Web3
from solcx import compile_source
#from web3.contract import ConciseContract

Using solc version v0.4.25


In [87]:
# Read in LUCE contract code
with open('./data/luce.sol', 'r') as file:
    contract_source_code = file.read()

In [88]:
# Compile & Store Compiled source code
compiled_sol = compile_source(contract_source_code)

In [89]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Dataset']

In [90]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

### Deploy

In [91]:
w3.eth.accounts[0]

'0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA'

In [92]:
# Set sender
w3.eth.defaultAccount = eth_account.address

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [93]:
# Create contract blueprint
Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

---

w3.version.api

At this moment we are unable to use a privately created fresh ethereum account for contract deployment.. When trying to deploy a contract with a new account we receive the error:
```
ValueError: {'message': 'sender account not recognized'
```
--> Researched and no easy way to use fresh accounts for contract deployment on Ganache (it IS possible to import existing accounts into Ganache to be used. But the whole idea is to create new accounts on the fly..)  
--> Use pre-funded Ganache accounts instead and allocate them to new users.


### Obtain Transcation Receipt

In [41]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [42]:
# Obtain address of freshly deployed contract
tx_receipt.contractAddress

'0x1Fe1b20f88e305d4F993Fb67457BcE7863D267f2'

## Second iteration:
Instead of creating new accounts for each user we will allocate a pre-funded ganache account to each user instead..

I want access to the Django user model from within the jupyter environment.  
To achieve that I used [this](https://stackoverflow.com/questions/35483328/how-to-setup-jupyter-ipython-notebook-for-django) information.

### Prepare Connection to Blockchain


In [3]:
from web3 import Web3

# Use Ganache as web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))
accounts = w3.eth.accounts

### Get Access to Django user

In [ ]:
import importlib.util
spec = importlib.util.spec_from_file_location("django.contrib.auth", "/path/to/file.py")

In [ ]:
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
foo.MyClass()

In [16]:
from django.contrib.auth import get_user_model
User = get_user_model()

ImproperlyConfigured: Requested setting AUTH_USER_MODEL, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

### Assign ethereum account

In [ ]:
from django.contrib.auth import get_user_model
User = get_user_model()
current_user =